In [ ]:
import cv2

# 1) Open a connection to the default camera (index 0).
#    If you have multiple cameras, you may need to use index 1, 2, etc.
camera = cv2.VideoCapture(0)

# 2) Read a frame from the camera.
ret, frame = camera.read()

if ret:
    # 3) If reading was successful, save the frame to a file.
    save_path = r"F:\Thesis\my_captured_image.jpg"  # raw string for Windows path
    cv2.imwrite(save_path, frame)
    print(f"Image saved to {save_path}")
else:
    print("Failed to capture image from camera.")

# 4) Release the camera resource.
camera.release()

In [ ]:
%matplotlib inline
import cv2
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from itertools import product
import tkinter as tk
from tkinter import filedialog

def test_parameters(img, param_ranges):
    """
    Test different parameter combinations and show intermediate steps, final results,
    and circle coordinates on the output image.
    """
    results = []
    
    # Create all combinations of parameters
    param_combinations = list(product(
        param_ranges['sigma'],
        param_ranges['clahe_limit'],
        param_ranges['param1'],
        param_ranges['param2'],
        param_ranges['size_divisor'],
        param_ranges['radius_factor']
    ))
    
    total_combinations = len(param_combinations)
    print(f"Testing {total_combinations} parameter combinations...")
    
    for idx, (sigma, clahe_limit, param1, param2, size_divisor, radius_factor) in enumerate(param_combinations, 1):
        # Preprocessing with current parameters
        if len(img.shape) == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        else:
            gray = img.copy()

        # Intermediate steps
        equalized = cv2.equalizeHist(gray)
        gaussian = ndimage.gaussian_filter(equalized, sigma=sigma)
        median_img = cv2.medianBlur(gaussian, 7)
        clahe = cv2.createCLAHE(clipLimit=clahe_limit, tileGridSize=(4, 4))
        preprocessed = clahe.apply(median_img)

        # Morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
        closed = cv2.morphologyEx(preprocessed, cv2.MORPH_CLOSE, kernel, iterations=2)
        opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel, iterations=1)
        clean_binary = cv2.medianBlur(opened, 5)

        # Detect wells with current parameters
        height, width = clean_binary.shape
        estimated_well_diameter = min(width, height) // size_divisor
        min_radius = int(estimated_well_diameter * radius_factor)
        max_radius = int(estimated_well_diameter * (radius_factor + 0.22))
        min_dist = int(estimated_well_diameter * 0.7)

        circles = cv2.HoughCircles(
            clean_binary,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=min_dist,
            param1=param1,
            param2=param2,
            minRadius=min_radius,
            maxRadius=max_radius
        )

        # Draw result
        result = img.copy()
        num_wells = 0
        if circles is not None:
            circles = np.uint16(np.around(circles[0]))
            # Sort circles by position
            circles = sorted(circles, key=lambda x: (x[1], x[0]))
            for i, (x, y, r) in enumerate(circles):
                # Draw circle
                cv2.circle(result, (x, y), r, (0, 255, 0), 2)
                # Mark the center
                cv2.circle(result, (x, y), 2, (0, 0, 255), 3)
                # Label with ID
                cv2.putText(result, f"ID {i+1}", (x - 20, y - r - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
                # Label with coordinates
                cv2.putText(result, f"({x}, {y})", (x - 20, y + r + 15),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0), 2)
                # Print coordinates in the console
                print(f"Circle {i+1}: x={x}, y={y}, r={r}")

            num_wells = len(circles)

            results.append({
                'params': {
                    'sigma': sigma,
                    'clahe_limit': clahe_limit,
                    'param1': param1,
                    'param2': param2,
                    'size_divisor': size_divisor,
                    'radius_factor': radius_factor
                },
                'num_wells': num_wells
            })

        # Create a figure for this parameter combination to show all intermediate steps
        fig, axes = plt.subplots(2, 4, figsize=(15, 10))
        axes = axes.ravel()

        # Show intermediate results
        axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original")
        axes[0].axis('off')

        axes[1].imshow(equalized, cmap='gray')
        axes[1].set_title("Equalized")
        axes[1].axis('off')

        axes[2].imshow(gaussian, cmap='gray')
        axes[2].set_title(f"Gaussian (σ={sigma})")
        axes[2].axis('off')

        axes[3].imshow(median_img, cmap='gray')
        axes[3].set_title("Median Blurred")
        axes[3].axis('off')

        axes[4].imshow(preprocessed, cmap='gray')
        axes[4].set_title(f"CLAHE (limit={clahe_limit})")
        axes[4].axis('off')

        axes[5].imshow(clean_binary, cmap='gray')
        axes[5].set_title("Morph")
        axes[5].axis('off')

        axes[6].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
        axes[6].set_title(f"Detected Wells: {num_wells}\n"
                          f"p1={param1}, p2={param2}\n"
                          f"sd={size_divisor}, rf={radius_factor}")
        axes[6].axis('off')

        # Hide the unused subplot (if any)
        axes[7].axis('off')

        fig.suptitle(f"Param combination {idx}/{total_combinations}", fontsize=6)
        plt.tight_layout()
        plt.show()

    # Print summary of results
    results.sort(key=lambda x: x['num_wells'], reverse=True)
    print("\nTop 5 parameter combinations:")
    for r in results[:5]:
        print(f"Number of wells: {r['num_wells']}")
        print("Parameters:", r['params'])
        print()

    return results

# Parameter ranges to test (adjust as needed)
param_ranges = {
    'sigma': [2],                    # Gaussian blur sigma
    'clahe_limit': [3],              # CLAHE clip limit
    'param1': [50],                  # Edge detection threshold for Hough
    'param2': [30],                  # Circle detection threshold for Hough
    'size_divisor': [3],             # Well size estimation divisor
    'radius_factor': [0.35]          # Radius calculation factor
}

# # Use Tkinter's file dialog to select the well plate image
# root = tk.Tk()
# root.withdraw()  # Hide the main window
# print("Please select the well plate image.")
# filename = filedialog.askopenfilename(title="Select the well plate image",
#                                       filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")])

# if filename:
#     print(f"Processing {filename}...")
#     img = cv2.imread(filename)
#     if img is not None:
#         results = test_parameters(img, param_ranges)
#     else:
#         print("Error reading the image file.")
# else:
#     print("No file selected.")

# Specify the image file path (update as needed)
filename = "F:/Thesis/trial/38.png"
print(f"Processing {filename}...")
img = cv2.imread(filename)
if img is not None:
    results = test_parameters(img, param_ranges)
else:
    print("Error reading the image file. Please check the file path.")


In [ ]:
import socket

def send_command(command, host='192.168.13.101', port=5000):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((host, port))
        s.sendall(command.encode('utf-8'))
        response = s.recv(1024).decode('utf-8')
        print(f"Response: {response}")

# Example commands
if __name__ == "__main__":
    send_command("HOME")
    send_command("MOVE_TO_COORDS 180 70 150")
    send_command("MOVE_TO_WELL A1")
    send_command("EXIT")